In [1]:
@file:DependsOn("org.jsoup:jsoup:1.17.1")
@file:DependsOn("com.github.doyaaaaaken:kotlin-csv-jvm:1.9.1")

import org.jsoup.Jsoup
import java.io.File
import java.io.InputStream
import java.net.HttpURLConnection
import java.net.URL
import com.github.doyaaaaaken.kotlincsv.dsl.csvReader
import com.github.doyaaaaaken.kotlincsv.dsl.csvWriter

val cookie = (System.getenv()["aoc_cookie"] ?: System.getProperty("aoc_cookie") ?: TODO("COOKIES!!!")).trim()

var year = 2025
var day = 1

class WrongAnswer : IllegalStateException("Not right answer")

fun createDirIfNotExist(dir: File) {
    if (!dir.exists()) {
        dir.mkdirs()
    }
}

fun readUrl(urlString: String): String {
    val con: HttpURLConnection = URL(urlString).openConnection() as HttpURLConnection
    con.setRequestMethod("GET")
    con.setRequestProperty("Cookie", cookie)
    con.setDoOutput(true)
    if (con.responseCode != 200) {
        throw IllegalStateException("${con.responseCode} - ${con.responseMessage}")
    }
    val content = String((con.content as InputStream).readAllBytes())
    return content
}

fun input(): List<String> {
    val file = File("input/$year/$day.txt")
    if (file.exists()) {
        return file.readLines()
    }
    createDirIfNotExist(File("input/$year"))
    val content = readUrl("https://adventofcode.com/${year}/day/${day}/input").removeSuffix("\n").split("\n")
    file.writeText(content.joinToString("\n"))
    return content
}

fun getInputs(part: Int): Pair<List<List<String>>, List<String>> {
    val doc = Jsoup.connect("https://adventofcode.com/$year/day/$day")
        .header("cookie", cookie)
        .get()
    val inputs = //doc.select("main > .day-desc pre code").map { it.text() }
        doc.select("main > .day-desc pre code").map { it.text() }
//            doc.select("main > .day-desc pre code:not(:has(em))").map { it.text() }
//                .ifEmpty { doc.select("main > .day-desc pre code").map { it.text() } }
    val answers = if (part == 1)
        doc.select("main > .day-desc:not(:has(#part2)) p > code > em").map { it.text() }
    else
        doc.select("main > .day-desc:has(#part2) p > code > em").map { it.text() }
    return inputs.map { it.split("\n") } to answers
}

data class Answer(val year: Int, val day: Int, val level: Int, val answer: String, val valid: Boolean) {
    companion object {
        operator fun invoke(columns: List<String>) =
            Answer(columns[0].toInt(), columns[1].toInt(), columns[2].toInt(), columns[3], columns[4].toBoolean())
    }

    fun toRow() = listOf(year, day, level, answer, valid).map { it.toString() }
}

fun getAnswers(): List<Answer> {
    val file = File("result.csv")
    return if (file.exists())
        csvReader().readAll(File("result.csv")).map { Answer(it) }
    else
        listOf()
}

object ConsoleColors {
    const val RESET = "\u001b[0m" // Text Reset
    const val RED = "\u001b[0;31m" // RED
    const val GREEN = "\u001b[0;32m" // GREEN
    const val YELLOW = "\u001b[0;33m" // YELLOW
}

object Logger {
    fun ok(msg: String) {
        println("${ConsoleColors.GREEN}✔${ConsoleColors.RESET} $msg")
    }

    fun info(msg: String) {

    }

    fun warn(msg: String, newLine: Boolean = false) {
        val line = "${ConsoleColors.YELLOW}?${ConsoleColors.RESET} $msg"
        if (newLine) println(line) else print(line)
    }

    fun error(msg: String) {
        println("${ConsoleColors.RED}✖${ConsoleColors.RESET} $msg")
    }
}

private fun sendAndValidate(year: Int, day: Int, level: Int, answer: String) {
    val con: HttpURLConnection =
        URL("https://adventofcode.com/$year/day/$day/answer").openConnection() as HttpURLConnection
    con.setRequestMethod("POST")
    con.setRequestProperty("Content-Type", "application/x-www-form-urlencoded")
    con.setRequestProperty("Cookie", cookie)
    con.setDoOutput(true)
    con.outputStream.use { os ->
        val input = "level=$level&answer=$answer".toByteArray()
        os.write(input, 0, input.size)
    }
    if (con.responseCode != 200) {
        throw IllegalStateException("${con.responseCode} - ${con.responseMessage}")
    }
    val content = String((con.content as InputStream).readAllBytes())
    when {
        "That's the right answer" in content ||
                "You don't seem to be solving the right level" in content -> {
//                Logger.ok("That's the right answer")
        }

        "Did you already complete it" in content -> {
//                Logger.warn("Did you already complete it")
            val problemText = readUrl("https://adventofcode.com/$year/day/$day")
            if ("<p>Your puzzle answer was <code>$answer</code>.</p>" !in problemText) {
                throw WrongAnswer()
            }
        }

        "That's not the right answer" in content -> {
//                Logger.error("That's not the right answer")
            throw WrongAnswer()
        }

        "You gave an answer too recently" in content -> {
            val res = Regex("You have (?:(\\d+)m )?(\\d+)s left to wait").find(content)!!
            fun String.parseInt() = if (isBlank()) 0 else toInt()
            var wait = res.groupValues[1].parseInt() * 60 + res.groupValues[2].parseInt()
            var msg = ""
            Logger.warn("", newLine = false)
            while (wait > 0) {
                val m = wait / 60
                val s = wait % 60
                msg =
                    "You have ${if (m > 0) "${m}m " else ""}${if (s > 0) "${s}s " else ""}left to wait (answer = $answer)"
                print(msg)
                Thread.sleep(1000L)
                wait--
                print("\b".repeat(msg.length) + " ".repeat(msg.length) + "\b".repeat(msg.length))
            }
            print("\b\b")
            sendAndValidate(year, day, level, answer)
        }
    }
}

fun send(level: Int, ans: String): Boolean {
    try {
        val answers = getAnswers().filter { it.year == year && it.day == day && it.level == level }
        answers.firstOrNull { it.answer == ans }?.let { return it.valid }

        val answer = try {
            sendAndValidate(year, day, level, ans)
            Answer(year, day, level, ans, true)
        } catch (e: WrongAnswer) {
            Answer(year, day, level, ans, false)
        }

        csvWriter().open(File("result.csv"), append = true) {
            writeRow(answer.toRow())
        }
        return answer.valid
    } catch (e: Throwable) {
        println(ans)
        throw e
    }
}

fun <T> parse(lambda: (List<String>) -> T): T {
    return lambda(input())
}

fun sendPart(level: Int, answer: Any) {
    try {
//            val input = (if (level == 1) parse(lines) else parse2(lines))
        val answer = answer.toString()
        val result = send(level, answer)
        if (result) {
            println("${ConsoleColors.GREEN}✔ ${answer}${ConsoleColors.RESET}")
        } else {
            println("${ConsoleColors.RED}✖${ConsoleColors.RESET} ${answer}")
        }
    } catch (e: Throwable) {
        e.printStackTrace()
    }
}

fun part1(result: Any) {
    sendPart(1, result)
}

fun part2(result: Any) {
    sendPart(2, result)
}


In [2]:
fun <T> Collection<T>.toCountMap(): Map<T, Int> {
    val result = mutableMapOf<T, Int>()
    forEach {
        result[it] = (result[it] ?: 0) + 1
    }
    return result
}

In [7]:
day = 1
val input = parse {
    it.map { l ->
        val k = l.take(1)
        val v = l.drop(1)
        k to v.toInt()
    }
}

run {
    val result = input.fold(0 to 50) { (res, cur), (k, v) ->
        val c = (cur + (if (k == "L") 1000000 - v else v)) % 100
        res + (if (c == 0) 1 else 0) to c
    }
    part1(result.first)
    val result2 = input.fold(0 to 50) { (res, cur), (k, v) ->
        (1..v).fold(res to cur) { (res, cur), _ ->
            val c = (cur + (if (k == "L") 1000000 - 1 else 1)) % 100
            res + (if (c == 0) 1 else 0) to c
        }
    }
    part2(result2.first)
}

✔ 1105
✔ 6599
